In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm

In [ ]:
def loadImage(file_path, read_param):
    param_dict = {'gray': cv2.IMREAD_GRAYSCALE, 'color': cv2.IMREAD_COLOR}#, 'with_alpha': cv2.IMREAD_UNCHANGED}
    ret = cv2.imread(file_path, param_dict[read_param])
    
    if ret is None:
        print('check file path')
        return

    if read_param == 'color':
        ret = cv2.cvtColor(ret, cv2.COLOR_BGR2RGB) 
        
    print('image', file_path, 'loaded')
    return ret

In [ ]:
def plotImages(imgs, titles=[], gray=False, normalise=True, fig_size=(10,10), num_cols = 2):
    if isinstance(gray, bool):
        gray = [gray for i in imgs]
    
    if isinstance(normalise, bool):
        normalise = [normalise for i in imgs]
        
    if not isinstance(imgs, list):
        imgs = [imgs]
    
    if not isinstance(titles, list):
        titles = [titles]
        
    num_rows = int(np.ceil(len(imgs)/num_cols))
    
    fig = plt.figure(fig_size)
    
    for i in len(imgs):
        ax = fig.add_subplot(num_rows, num_cols, i+1)
        
        img = imgs[i]
        vmin = 0
        vmax = 255
        if gray[i] and len(img.shape)==3: 
            img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        
        if normalised[i]:
            vmin = img.min()
            vmax = img.max()
        
        if gray[i]:
            ax.imshow(img, cmap='gray', vmin=vmin, vmax = vmax)
        else:
            ax.imshow(img, vmin=vmin, vmax=vmax)
        
        

In [ ]:
def blurKernel(r, theta):
    kernel = np.zeros((r,r))
    kernel[r//2,:] = 1/r
    return rotateMatrix(kernel, theta)

In [ ]:
def blurLegend(directions, magnitude):
    norm = mpl.colors.Normalize(0, np.pi)
    plt.imshow(norm(directions))
    plt.show()

In [ ]:
def rotateMatrix(matrix, angle):
    from skimage.transform import rotate
    return rotate(matrix, angle)

In [ ]:
def MSELoss(img1, img2):
    return np.sum(np.power(img1 - img2, 2))

In [ ]:
def linearBlur(img, r, theta):
    kernel = blurKernel(r, theta)
    return cv2.filter2D(np.uint8(img), -1, kernel)

In [ ]:
def nonUniformBlurMatrix(img):
    K = []
    for i in tqdm(range(img.shape[0])):
        for j in range(img.shape[1]):
            r = np.random.randint(1, 1+min(2*(img.shape[0]-1-i)+1, 2*(img.shape[1]-1-j)+1, 30, 2*i+1, 2*j+1))
            
            theta = np.random.randint(0,180)
            kernel = blurKernel(r,theta)

            row = np.zeros(img.shape[0]*img.shape[1])

            idxs, _ = np.meshgrid(np.arange(0,r), np.arange(0,r))
            idxs = idxs + (np.arange(0,r)*img.shape[1])[:,None] + (i-r//2)*img.shape[1] + j-r//2
            idxs = idxs.ravel()

            row[idxs] = kernel.ravel()
            K.append(row)
            
    return np.array(K)
            
            
def nonUniformBlur(img):
    K = nonUniformBlurMatrix(img)
    return (K @ img.ravel()[:,None]).reshape(img.shape), K

In [ ]:
img = loadImage('./resources/building.jpg', 'gray')
np.set_printoptions(threshold=10000000000)

blurred_img, K = nonUniformBlur(img)
plt.imshow(blurred_img)
plt.figure()
plt.imshow(img)
plt.show()